In [1]:
seed = 42

import random
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

random.seed(seed)
TFHUB_CACHE_DIR = r'C:/Data/tfhub'

In [2]:
# Reduce logging output.
logging.set_verbosity(logging.ERROR)

df = pd.read_csv('nlp-getting-started/train_clean_spellcheck.csv')
train_df = df.sample(frac=1).reset_index(drop=True)[['id','text','target']]
test_df = pd.read_csv('nlp-getting-started/test_clean_spellcheck.csv')[['id','text']]
train_df.head()

,id,text,target
0,9525,abstention Zionist terrorist demolish tire rep...,1
1,8572,mi not home i need to watch screams,0
2,2487,monkey just collided heads with our ninja comm...,0
3,3849,ignition knock detonation sensor interconnecti...,0
4,3727,i always felt like the ramekins were black peo...,0


In [3]:
test_df.head()

,id,text
0,0,just happened a terrible car crash
1,2,heard about earthquake is different cities sta...
2,3,there is a forest fire at spot pond geese are ...
3,9,apocalypse lighting Spokane wildfires
4,11,typhoon counselor kills 28 in china and Taiwan


In [4]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df["target"], num_epochs=None, shuffle=True, target_column='prediction')

# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df["target"], shuffle=False, target_column='prediction')

# Prediction on the test data set.
predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
    test_df, shuffle=False, target_column='target')

In [5]:
embedded_text_feature_column = hub.text_embedding_column(
    key="text", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")

UnknownError: Failed to rename: C:\Users\vicxjfn\AppData\Local\Temp\tfhub_modules\32f2b2259e1cc8ca58c876921748361283e73997.542fcd2c01e34c20a32a5894acff042d.tmp to: C:\Users\vicxjfn\AppData\Local\Temp\tfhub_modules\32f2b2259e1cc8ca58c876921748361283e73997 : Access is denied.
; Input/output error

In [ ]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[500, 100],
    feature_columns=[embedded_text_feature_column],
    n_classes=2,
    optimizer=tf.train.AdagradOptimizer(learning_rate=0.003))

In [ ]:
# Training for 200 steps means 25,600 training examples with the default
# batch size. This is roughly equivalent to 5 epochs since the training dataset
# contains 5,000 examples.
estimator.train(input_fn=train_input_fn, steps=200);

In [ ]:
train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)

print("Training set accuracy: {accuracy}".format(**train_eval_result))

In [ ]:
test_predict_result = estimator.predict(input_fn=predict_test_input_fn)

In [ ]:
def get_predictions(estimator, input_fn):
  return [x["class_ids"][0] for x in estimator.predict(input_fn=input_fn)]

LABELS = [
    "negative", "positive"
]

# Create a confusion matrix on training data.
with tf.Graph().as_default():
  cm = tf.confusion_matrix(train_df["target"], 
                           get_predictions(estimator, predict_train_input_fn))
  with tf.Session() as session:
    cm_out = session.run(cm)

# Normalize the confusion matrix so that each row sums to 1.
cm_out = cm_out.astype(float) / cm_out.sum(axis=1)[:, np.newaxis]

sns.heatmap(cm_out, annot=True, xticklabels=LABELS, yticklabels=LABELS);
plt.xlabel("Predicted");
plt.ylabel("True");

In [ ]:
# Prediction on the test set.
predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
    test_df, shuffle=False, target_column='prediction')

predictions = estimator.predict(input_fn=predict_train_input_fn)

In [ ]:
labels = [0, 1]
output = [x for x in test_predict_result]
test_df['target'] = [labels[np.argmax(item['probabilities'])] for item in output]

In [ ]:
test_df[['id','target']].to_csv('nlp-getting-started/submission_6.csv',index=False)